<a href="https://colab.research.google.com/github/tomonari-masada/course2024-sml/blob/main/11_document_clustering_(in_class).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# クラスタリング
* クラスタリングの代表的な手法であるk平均法を使ってみる。
* ついでに、言語モデルを使ったテキストマイニングを体験してみる。

## 例題: 文書クラスタリング

* Transformerベースの日本語対応言語モデルを使って、テキストのベクトル表現を得る。
  * Transformerというニューラルネットワークについては、いずれ学びます。
  * 有名な解説記事 https://jalammar.github.io/illustrated-transformer/
* テキストをベクトルとして表現することを「embedする」と言う。
  * embedすることで得られるベクトルのことを「embedding」と言う。
* そして、テキストのembeddingをk平均法でクラスタリングする。

* ランタイムのタイプをGPUにしておく。

## インストール

### spaCyの日本語モデル

* 日本語テキストを形態素解析するために使う。
  * インストール後に「セッションの再起動」が必要。

In [1]:
!python -m spacy download ja_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 10.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ja_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Hugging Faceのdatasetsライブラリ

* ライブドアニュースコーパスを取得するために使う。

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

### SentenceTransformersライブラリ
* 言語モデルを使ってテキストを埋め込む際に便利なライブラリ。
  * https://sbert.net/index.html

In [2]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

## インポート

In [3]:
from tqdm.auto import tqdm
import collections
import numpy as np

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import spacy

from datasets import load_dataset
from transformers import set_seed
from sentence_transformers import SentenceTransformer

# 再現性の確保
set_seed(1234)

## データセット
* livedoorニュースコーパスを使う。

In [4]:
dataset = load_dataset(
  "shunk031/livedoor-news-corpus",
  train_ratio=0.8, val_ratio=0.1, test_ratio=0.1,
  random_state=42,
  shuffle=True,
  trust_remote_code=True,
)

num_categories = len(set(dataset["train"]["category"]))

category_names = [
  'movie-enter',
  'it-life-hack',
  'kaden-channel',
  'topic-news',
  'livedoor-homme',
  'peachy',
  'sports-watch',
  'dokujo-tsushin',
  'smax',
]

print(f"num_categories: {num_categories}")
print(f"category_names: {category_names}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

num_categories: 9
category_names: ['movie-enter', 'it-life-hack', 'kaden-channel', 'topic-news', 'livedoor-homme', 'peachy', 'sports-watch', 'dokujo-tsushin', 'smax']


In [5]:
dataset["train"][0]

{'url': 'http://news.livedoor.com/article/detail/6537176/',
 'date': '2012-05-08T10:55:00+0900',
 'title': 'ヱヴァスマホ「SH-06D NERV」に同梱される専用リアカバーが公開！特設サイトがリニューアル',
 'content': '専用リアパネルが2枚同梱！  カラーは2日、NTTドコモから6月に発売が予定されているAndroidスマートフォン「docomo NEXT series SH-06D NERV」のヱヴァンゲリヲン劇場版特設サイト「docomo NEXT series SH-06D NERV : SH-06D NERV特設サイト」を更新し、リアカバーが2枚同梱するということが新たに判明しています。  SH-06D NERVはシャープ製Androidスマートフォン「AQUOS PHONE SH-06D」をベースに、今秋公開予定の映画「ヱヴァンゲリヲン 新劇場版：Q」とコラボレーションしたモデルです。以前にヱヴァンゲリヲンとのコラボレーションしたケータイがありましたが、ヱヴァンゲリヲンコラボモデルとしては第2弾で、初のスマートフォンになります。  今回リニューアルしたのは、NTTドコモが運営する特設サイト「docomo NEXT series SH-06D NERV : SH-06D NERV特設サイト | 製品 | NTTドコモ」ではなく、ヱヴァンゲリヲン劇場版を制作しているカラーが運営する特設サイトです。  リニューアルされたことで、従来公開されていた専用リアカバー「TYPE:NERV」に加え、同梱される2枚目のリアカバー「TYPE:MAGI」があることが明らかとなっています。   TYPE:NERV  TYPE:MAGI  また、SH-06DにはNOTTVのアンテナを搭載した卓上ホルダーが同梱されていますが、SH-06D NERVにも同等の卓上ホルダーが同梱され、こちらもNERV専用の特別仕様となっていることが公開されています。    この特設サイトでは、今後、順次、発売までに搭載コンテンツなども公開されていくようですので、興味のある人は是非チェックしてみてください。  記事執筆：memn0ck  ■関連リンク ・エスマックス（S-MAX） ・

In [8]:
dataset["train"]["title"][:10]

['ヱヴァスマホ「SH-06D NERV」に同梱される専用リアカバーが公開！特設サイトがリニューアル',
 '男のマザコン、許せる？\u3000許せない？',
 'ついに発売開始！小さな箱は3段重ね！？クアッドコアCPU搭載スマホ「ARROWS X F-10D」を開封してみた【レビュー】',
 '目指せ！\u3000“水周りピカピカ女子”\u3000激泡クリーナー「スクラビングバブル」でラクラクお掃除',
 '映画のセリフに学ぶ！ストレスフリーな「スローラブ」のススメ',
 'mixiの新サービスに疑問の声？ mixiのTwitterから友達をさがす機能が不評',
 '無料PDF作成ソフトを大活用！PDFでビジネスが変わる【知っ得！虎の巻】',
 '野郎ども、バグにツイートをくれてやれ！『スターシップ・トゥルーパーズ：インベイジョン』キャンペーンが開始',
 '全世界63ヶ国で1位獲得『ワイルド・スピード MEGA MAX』が早くもブルーレイ＆DVDに',
 '【韓国ニュース】日本大使館トラック突入事件の容疑者が日本人活動家に“公開提案”']

In [7]:
dataset["train"]["content"][0]

'専用リアパネルが2枚同梱！  カラーは2日、NTTドコモから6月に発売が予定されているAndroidスマートフォン「docomo NEXT series SH-06D NERV」のヱヴァンゲリヲン劇場版特設サイト「docomo NEXT series SH-06D NERV : SH-06D NERV特設サイト」を更新し、リアカバーが2枚同梱するということが新たに判明しています。  SH-06D NERVはシャープ製Androidスマートフォン「AQUOS PHONE SH-06D」をベースに、今秋公開予定の映画「ヱヴァンゲリヲン 新劇場版：Q」とコラボレーションしたモデルです。以前にヱヴァンゲリヲンとのコラボレーションしたケータイがありましたが、ヱヴァンゲリヲンコラボモデルとしては第2弾で、初のスマートフォンになります。  今回リニューアルしたのは、NTTドコモが運営する特設サイト「docomo NEXT series SH-06D NERV : SH-06D NERV特設サイト | 製品 | NTTドコモ」ではなく、ヱヴァンゲリヲン劇場版を制作しているカラーが運営する特設サイトです。  リニューアルされたことで、従来公開されていた専用リアカバー「TYPE:NERV」に加え、同梱される2枚目のリアカバー「TYPE:MAGI」があることが明らかとなっています。   TYPE:NERV  TYPE:MAGI  また、SH-06DにはNOTTVのアンテナを搭載した卓上ホルダーが同梱されていますが、SH-06D NERVにも同等の卓上ホルダーが同梱され、こちらもNERV専用の特別仕様となっていることが公開されています。    この特設サイトでは、今後、順次、発売までに搭載コンテンツなども公開されていくようですので、興味のある人は是非チェックしてみてください。  記事執筆：memn0ck  ■関連リンク ・エスマックス（S-MAX） ・エスマックス（S-MAX） smaxjp on Twitter ・docomo NEXT series SH-06D NERV : SH-06D NERV特設サイト ・docomo NEXT series SH-06D NERV | 製品 | NTTドコモ'

## 多言語E5による埋め込み

* Multilingual E5を使う。
  * テキストのembeddingにおいて優れている言語モデル。
  * 論文 https://arxiv.org/abs/2402.05672
  * Hugging Face https://huggingface.co/intfloat/multilingual-e5-large-instruct

* 参考: テキスト埋め込みのleaderboard
  * https://huggingface.co/spaces/mteb/leaderboard

* SentenceTransformerを使ったテキストの埋め込みについては、下のWebページを参照。
  * https://sbert.net/examples/applications/computing-embeddings/README.html

In [9]:
model_id = "intfloat/multilingual-e5-large-instruct"
model = SentenceTransformer(model_id)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

* 試しに、一つだけ、テキストを埋め込んでみる。

In [10]:
dataset["train"][0]["title"]

'ヱヴァスマホ「SH-06D NERV」に同梱される専用リアカバーが公開！特設サイトがリニューアル'

In [11]:
model.encode(dataset["train"][0]["title"])

array([-0.00018783,  0.00825893, -0.00325229, ..., -0.0211348 ,
       -0.03856956,  0.01676247], dtype=float32)

In [12]:
type(model.encode(dataset["train"][0]["title"]))

numpy.ndarray

* ライブドアニュースコーパスの全タイトルを埋め込む。

In [13]:
embeddings = model.encode(dataset["train"]["title"], show_progress_bar=True)

Batches:   0%|          | 0/185 [00:00<?, ?it/s]

* 埋め込みは普通にNumPyの配列として得られている。

In [14]:
type(embeddings)

numpy.ndarray

In [15]:
embeddings.shape

(5894, 1024)

* 全記事内容を埋め込むには以下のようにする。  
  * RTX3080搭載PCを使うと1分で終わる。

In [ ]:
#content_embeddings = model.encode(dataset["train"]["content"], show_progress_bar=True)

* ただし、どのテキストも先頭から512トークンで切られていることに注意。
  * 長いテキストは、途中までの内容しかembeddingに反映されない。
  * それでも、分類やクラスタリングがうまくいくことも多い。

In [16]:
model.max_seq_length

512

* トークン数の調べ方
  * トークナイザにテキストを分割させる。
  * 分割によって得られたトークンの個数を数える。

In [ ]:
model.tokenize([dataset["train"][0]["title"]])

In [ ]:
(model.tokenize([dataset["train"][0]["title"]])['input_ids']).shape[1]

* 埋め込みを保存。

In [ ]:
with open('embeddings.npy', 'wb') as f:
  np.save(f, embeddings)

In [ ]:
#with open('content_embeddings.npy', 'wb') as f:
#  np.save(f, content_embeddings)

* 読み込みは以下のようにする。

In [ ]:
with open('embeddings.npy', 'rb') as f:
  embeddings = np.load(f)

In [ ]:
#with open('content_embeddings.npy', 'rb') as f:
#  content_embeddings = np.load(f)

## クラスタのラベリングに使う単語の抽出

* 全テキストを形態素解析する。
  * 形態素解析＝単語への分割

In [17]:
nlp = spacy.load("ja_core_news_sm")
corpus = []
for text in tqdm(dataset["train"]["title"]):
  corpus.append(" ".join([token.lemma_ for token in nlp(text)]))

  0%|          | 0/5894 [00:00<?, ?it/s]

In [20]:
dataset["train"][0]["title"]

'ヱヴァスマホ「SH-06D NERV」に同梱される専用リアカバーが公開！特設サイトがリニューアル'

In [19]:
corpus[0]

'ヱヴァスマホ 「 SH - 06 d nerv 」 に 同梱 する れる 専用 リアカバー が 公開 ! 特設 サイト が リニューアル'

* scikit-learnでTF-IDFを計算する。
* `TfidfVectorizer`の`min_df`パラメータは適当に調節する。
  * クラスタのラベリングに向かないマイナーな単語が含まれないようにする。

In [21]:
vectorizer = TfidfVectorizer(min_df=10)
X_train = vectorizer.fit_transform(corpus).toarray()
vocab = np.array(vectorizer.get_feature_names_out())

In [22]:
vocab.size

996

In [23]:
print(list(vocab))

['01', '02', '04', '05', '06', '07', '08', '09', '10', '100', '1000万', '101', '11', '12', '13', '14', '15', '16', '17', '18', '1日', '20', '2010', '2011', '2012', '21', '22', '23', '24', '25', '27', '29', '2人', '30', '40', '48', '50', '500', 'akb', 'android', 'anteprima', 'aquos', 'arrows', 'au', 'by', 'cafe', 'cm', 'cpu', 'dena', 'dvd', 'eluga', 'excel', 'facebook', 'fi', 'for', 'galaxy', 'ghz', 'google', 'gx', 'hd', 'ht', 'htc', 'ics', 'iii', 'in', 'ios', 'ipad', 'iphone', 'is', 'isw', 'it', 'kddi', 'lan', 'led', 'lte', 'mac', 'medias', 'nhk', 'note', 'nottv', 'ntt', 'office', 'ok', 'one', 'optimus', 'os', 'paul', 'pc', 'peachy', 'phone', 'play', 'presented', 'pro', 'regza', 'report', 'salon', 'sc', 'sh', 'siii', 'sim', 'smith', 'sns', 'so', 'sports', 'ssd', 'sx', 'tab', 'tbs', 'the', 'tv', 'twitter', 'ultrabook', 'up', 'usb', 'vol', 'vs', 'watch', 'web', 'wi', 'wifi', 'wimax', 'windows', 'word', 'w杯', 'xi', 'xperia', 'あなた', 'あの', 'あり', 'ある', 'いい', 'いう', 'いく', 'いける', 'いちおう', 'いつ', 'いつ

In [28]:
X_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.40928076,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [25]:
embeddings[0]

array([-0.00018784,  0.00825894, -0.0032523 , ..., -0.02113478,
       -0.03856954,  0.01676248], dtype=float32)

## ラベリング用単語の埋め込み

* 各単語について、その単語を含むテキストの埋め込みベクトルの加重平均を求める。
* 加重平均の重みは、各テキストにおけるその単語のTF-IDFの値を使って定める。

In [29]:
X_train.sum(0).shape

(996,)

* 「公開」という単語のtf-idfが、文書12で0.1, 文書254で0.3, 文書4032で0.6とする。
* 「公開」のembeddingを、以下のように計算する。
$0.1 \times \mathbf{x}_{12} + 0.3 \times \mathbf{x}_{254} + 0.6 \times \mathbf{x}_{4032}$


In [32]:
# それぞれの単語について、和が1になるように規格化
text_weights = X_train / X_train.sum(0)

In [31]:
vocab_embeddings = np.dot(text_weights.T, embeddings)

## テキストの埋め込みのクラスタリング



### k-平均法によるクラスタリング

In [33]:
n_clusters = 20
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(embeddings)
#kmeans.fit(content_embeddings) # 本文の場合はこちら。
centers = kmeans.cluster_centers_

* クラスタの重心を保存。

In [ ]:
with open(f'centers_{n_clusters}.npy', 'wb') as f:
  np.save(f, centers)

In [ ]:
with open(f'centers_{n_clusters}.npy', 'rb') as f:
  centers = np.load(f)

### クラスタのサイズを調べる

* クラスタのインデックスをキーとし、そのサイズを値とする辞書を作る。

In [36]:
kmeans.labels_

array([11,  9, 11, ..., 13, 15,  0], dtype=int32)

In [37]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
size_dict = dict(zip(unique, counts))

* 辞書のエントリを、キーではなく値でソートする。

In [38]:
print([sorted(size_dict.items(), key=lambda item: item[1], reverse=True)])

[[(7, 449), (0, 405), (18, 394), (8, 388), (16, 368), (17, 356), (15, 345), (10, 339), (12, 330), (11, 318), (1, 288), (4, 279), (3, 259), (5, 258), (14, 253), (9, 235), (2, 188), (19, 160), (13, 145), (6, 137)]]


## クラスタのラベリング
* 各クラスタの重心に近い単語でラベリングする。

* テキストの埋め込みは、長さ1のベクトルになっている。

In [39]:
np.linalg.norm(embeddings, axis=-1)

array([1.        , 0.99999994, 1.        , ..., 0.99999994, 0.99999994,
       1.        ], dtype=float32)

In [40]:
np.linalg.norm(centers, axis=-1)

array([0.94010097, 0.91801184, 0.9345086 , 0.92599636, 0.92983425,
       0.92901224, 0.9241918 , 0.918104  , 0.9134064 , 0.9166613 ,
       0.92441803, 0.93504876, 0.9236237 , 0.92531514, 0.91995716,
       0.9199063 , 0.92426914, 0.92824906, 0.9129726 , 0.92651063],
      dtype=float32)

* テキストとラベリング用の単語との類似度はコサイン類似度で測る。

In [41]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vocab_embeddings, centers)

* 重心に近い順に30個の単語を表示する。

In [42]:
for i in range(similarities.shape[-1]):
  indices = np.argsort(- similarities[:,i])
  print(vocab[indices[:30]])

['watch' 'sports' '選手' '試合' '監督' '代表' 'なでしこ' 'プロ' '野球' '巨人' '語る' 'です' '岡田'
 'てる' 'サッカー' 'w杯' '星野' 'さん' '明かす' '言う' '告白' 'チーム' '優勝' '松井' 'ノム' '開幕' '斎藤'
 'スポーツ' '桑田' '本田']
['キレイ' '美容' '目指す' '入れる' '美肌' '美人' '作る' 'なれる' '効果' 'ケア' 'ビューティ' '健康' 'ゲット'
 'なる' '簡単' 'ため' 'まで' 'アイテム' 'すぐ' 'ます' 'こそ' 'する' 'ビューティー' '秘訣' 'たい' '取る'
 'ボディ' '人気' '美しい' 'できる']
['提供' '開始' 'ntt' '更新' 'ドコモ' 'ソフトウェア' '向け' 'および' '具合' 'phone' 'kddi'
 'バージョン' '01' 'au' 'ソフトバンク' '起動' 'スマートフォン' '対応' '02' 'にて' '端末' '予定' 'sh'
 'lte' 'モード' 'モバイル' '利用' '04' 'sc' 'ケータイ']
['選手' '代表' '監督' 'です' '言う' 'チーム' '明かす' 'さん' 'てる' '試合' '語る' 'ファン' 'watch'
 'sports' '日本' '報道' 'サッカー' 'プロ' '五輪' '野球' '言及' 'なでしこ' '怒る' 'いく' 'やる' '告白'
 'ない' '対する' 'ある' '発言']
['アプリ' 'android' 'iphone' 'チャンス' '使う' 'for' 'やすい' 'オススメ' '便利' '掴める' '使える'
 '無料' '楽しめる' '機能' 'できる' '情報' 'ゲーム' 'リリース' 'みる' 'アップ' '手軽' 'すすめ' '操作' 'スマホ'
 '専用' '紹介' '試す' '感覚' '配信' '満載']
['映画' 'まとめ' '読み' '物語' '週末' '作品' '上映' '主演' 'ストーリー' '編集' '批評' 'オトナ' '描く'
 '公開' '女優' '観る' '特別' '映像' '最高' 'ドラマ' 'dvd' 'プレミア' 'vs' 

# 課題
* それぞれのクラスタについて、重心に近い元々のテキスト（つまり記事タイトル）を5件ずつ表示させてみよう。
* それらのテキストの内容に、上で得たラベルが合っているかどうか、確かめよう。